In [11]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import time
import pydde as d

In [12]:
#Parameters
samplenum = 1
epochs = 200
input_size = 3
output_size = 3
learning_rate = 0.01
time_length = 3; #seconds

In [13]:
# Generate simulation
dyn = d.PyDyn('../Data/point-mass_pendulum.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)

#Sample targets only variables in z direction
y_target = np.zeros((samplenum, 3))
y_target[:,2] = np.random.rand(samplenum)
y_target[:,1] = 2
print(y_target)

p = dyn.get_p(y_target.transpose(), dyn.p_init)
y_target= torch.tensor(y_target, requires_grad= True)

[[0.         2.         0.19606366]]
float64


## Building the custon Simulation Activation Function

In [14]:
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        #print(f'input: {input.shape}')
        p = input.clone().numpy().transpose()
        state = dyn.compute(p)
        y_pred = torch.tensor(state.y[-3:], requires_grad = True)
        #print(f'y_pred: {y_pred.dtype}')
        
        ctx.save_for_backward(input)
        
        return y_pred, input
    
    @staticmethod
    def backward(ctx, grad_output, input):
        #print(grad_output.shape)
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        state= dyn.compute(p)
        dy_dp = dyn.dy_dp(state, p)
        dy_dp = dy_dp[-3:, :]
        print(f'shape of dy/dp: {dy_dp.shape}')
        print(f'shape of grad_output: {grad_output.shape}')
        grad_output = grad_output.unsqueeze(0).t()
        print(f'shape of grad_output unsqueezed: {grad_output.shape}')
        
        grad_input = torch.tensor(dy_dp, requires_grad = True).t().mm(grad_output).t()
        #grad_input = torch.tensor(dy_dp, requires_grad = True)
        print(f'shape of grad_input: {grad_input.dtype}')

        return grad_input, None

Simulate = Simulate.apply
class ActiveLearn(nn.Module):

    def __init__(self, n_in, out_sz):
        super(ActiveLearn, self).__init__()

        self.L_in = nn.Linear(n_in, 3*time_length).double()
        self.Relu = nn.ReLU(inplace=True).double()
        self.P = nn.Linear(3*time_length, 3*time_length).double()
        #self.L_out = nn.Linear(3, 3)
    
    def forward(self, input):
        x = self.L_in(input)
        x = self.Relu(x)
        x = self.P(x)
        x, p = Simulate(x)
        #x = self.L_out(x)
        return x, p
    
model = ActiveLearn(input_size, output_size)



## FE Test for custom function

In [15]:
# Error for whole simulation
from numpy import linalg as LA

FE = 1e-6
p = torch.tensor(p, requires_grad = True).double()
y, p0 = Simulate(p)
#test = sum(y)
#test.backward()
y.backward(torch.FloatTensor([1.0, 1.0, 1.0]).double())
dy_dp = p.grad.double()
print(f'dy_dp shape = {p.grad.shape}')
print(f'dy_dp = {dy_dp}')
dy_dp_FD = np.zeros((1,len(dyn.p_init)))
#dy_dp_FD = np.zeros((3,len(dyn.p_init)))

for i in range(len(dyn.p_init)):
    dp= np.zeros(len(dyn.p_init))
    dp[i] = FE
    dp = torch.tensor(dp)
    y_p, pp = Simulate(p + dp)
    y_m, pm = Simulate(p - dp)
    y_p = y_p.detach().numpy()
    y_m = y_m.detach().numpy()
    dy_dp_FD[0, i] = sum((y_p - y_m) / (2* FE))
    #dy_dp_FD[:, i] = (y_p - y_m) / (2* FE)


print(f' dy_dp_FD = {dy_dp_FD}')
print(f' dy_dp_FD = {dy_dp_FD.shape}')
dy_dp = dy_dp.detach().numpy()
err = LA.norm(dy_dp_FD - dy_dp)
print('The error is: 'err)

shape of dy/dp: (3, 9)
shape of grad_output: torch.Size([3])
shape of grad_output unsqueezed: torch.Size([3, 1])
shape of grad_input: torch.float64
dy_dp shape = torch.Size([9])
dy_dp = tensor([ 0.0079, -0.0581,  0.0079,  0.0058,  0.1788,  0.0062,  0.0028,  0.9026,
         0.0051], dtype=torch.float64)
 dy_dp_FD = [[ 0.00793809 -0.05814328  0.00789857  0.005836    0.17882516  0.00617314
   0.00284413  0.90260386  0.00507941]]
 dy_dp_FD = (1, 9)
6.688302177514362e-10


## FD Test for dy_dp Function

In [16]:
from numpy import linalg as LA

#Calculate dy_dp with FE
FE = 1e-6
dy_dp = dyn.dy_dp(state_init, dyn.p_init)
dy_dp_FD = np.zeros((len(dyn.p_init),len(dyn.p_init)))

for i in range(len(dyn.p_init)):
    dp= np.zeros(len(dyn.p_init))
    dp[i] = FE
    pos = dyn.p_init + dp
    y_p = dyn.compute(pos)
    y_m = dyn.compute(dyn.p_init - dp)
    dy_dp_FD[:, i] = (y_p.y - y_m.y) / (2* FE)
err = LA.norm(dy_dp_FD - dy_dp)
print(err)


2.5942135105905507e-06
